**Because our datafile is so large we need to use git lfs before cloning the file**

In [ ]:
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
!sudo apt-get install git-lfs
!git lfs install

**clone the repo to access the data files**

In [ ]:
!git clone https://github.com/Cbmunns/Traffic_Sign_Recognition.git

**Unzip the large file**


In [ ]:
import zipfile

!unzip /content/Traffic_Sign_Recognition/sign_data.zip

**Import important libraries for the file**

In [8]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from PIL import Image
import os, sys
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout

**Here we set up the lists to hold the data and their respective labels**

**Set the number of classes or different number of signs we wish to train on**

**Then set the path for or data set**

In [ ]:
data = []
labels = []
classes = 43
os.chdir("/content")
cur_path = os.getcwd()
print(cur_path)

**Then iterating through each folder, go over every image**

**Open the image**

**Resize each picture to 30 by 30 for consistency**

**Add it to the image list**

**And then add it's corresponding label**

In [ ]:
#Retrieving the images and their labels 
for i in range(classes):
    path = os.path.join(cur_path,'train', str(i))
    print(path)
    images = os.listdir(path)
    #print(images)
    for a in images:
        try:
            image = Image.open(path + '/'+ a)
            image = image.resize((30,30))
            image = np.array(image)
            data.append(image)
            labels.append(i)
        except:
            print("Error loading image")


**Convert the lists into numpy arrays**

In [12]:
data = np.array(data)
labels = np.array(labels)

**We then get the dimensions of the arrays**

**Then seperate into the test/train sets**

In [ ]:
print(data.shape, labels.shape)
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

**Place a quick check on the dimensions of the new arrays**

In [ ]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

**We then use one hot encoding to break the different labels into easily deciphered binary**

For ex.)  
**label**  
stop,  
10,  
20,  

Becomes.)  
**stop  10  20**  
1, 0, 0,  
0, 1, 0,  
0, 0, 1,  




In [15]:
y_train = to_categorical(y_train, 43)
y_test = to_categorical(y_test, 43)

**This code is to identify and verify that a GPU is available and what it's label is**


In [ ]:

physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("Num GPUS Available: ", len(physical_devices))
tf.config.experimental.set_memory_growth(physical_devices[0], True)

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

**Using GPU acceleration we build our model**



In [ ]:
with tf.device('/gpu:0'): 
  #Building the model
  model = Sequential()
  # We establish our convolutional layers with filter size of 5X5
  model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu', input_shape=X_train.shape[1:]))
  model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu'))
  # Max-pooling/down sample the output of this data  of size 2 x 2
  model.add(MaxPool2D(pool_size=(2, 2)))
  # Improve the reliability we use dropout to regularize the output and help prevent overfitting
  model.add(Dropout(rate=0.25))
  # We repeat! now with the filter set to 3 X 3
  model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
  model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
  # Down sample
  model.add(MaxPool2D(pool_size=(2, 2)))
  # Regularize
  model.add(Dropout(rate=0.25))
  # Flatten the data to prepare for the Dense layer
  model.add(Flatten())
  # Run this through a standard Dense layer
  model.add(Dense(256, activation='relu'))
  # Regularize
  model.add(Dropout(rate=0.5))
  # Final output layer with all 43 classes represented
  model.add(Dense(43, activation='softmax'))

  # Compile the model
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

  # Fit and store the model with 15 epochs
  history = model.fit(X_train, y_train, batch_size=32, epochs= 15, validation_data=(X_test, y_test))
  

**Using the history object we observe the accuracy of our model over time**

In [ ]:
plt.figure(0)
plt.plot(history.history['accuracy'], label='training accuracy')
plt.plot(history.history['val_accuracy'], label='val accuracy')
plt.title('Accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()
plt.show()

**Again using history we examine the loss of the model**

In [ ]:
plt.figure(1)
plt.plot(history.history['loss'], label='training loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.title('Loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

**Using a premade CSV we access shuffled photos that haven't been ran on the model to test true accuracy**

In [ ]:
from sklearn.metrics import accuracy_score

y_test = pd.read_csv('Test.csv')
labels = y_test["ClassId"].values
imgs = y_test["Path"].values
data=[]
for img in imgs:
    image = Image.open(img)    
    image = image.resize((30,30))    
    data.append(np.array(image))

X_test=np.array(data)
pred = model.predict_classes(X_test)


**Output the true accuracy of the model**

In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(labels, pred))


**Base code was used as a learning tool and was sourced from**  
https://data-flair.training/blogs/python-project-traffic-signs-recognition/